In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# read_data and make train and test data for example
# https://www.kaggle.com/datasets/arnabchaki/data-science-salaries-2023
def read_data():
    df = pd.read_csv("ds_salaries.csv")
    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
    return df_train.reset_index(drop=True), df_test.reset_index(drop=True)

df_train, df_test = read_data()
print(df_train.shape, df_test.shape)
df_train

In [ ]:
# make baseblock for funciton. Update by class inheritance.
class base():
    def fit(self, input_df: pd.DataFrame):
        return self.transform(input_df)
    def transform(self, input_df: pd.DataFrame)->pd.DataFrame:
        raise NotImplementedError()

In [ ]:
## 要修正
# # 時間特徴量の簡易処理（あれば。事前に時間特徴量にparse_datesで変換しておく。）
# # 時間を分解
# def sep_time(df, col, col2=0):
#   df[col+"_y"]=df[col].dt.year
#   df[col+"_m"]=df[col].dt.month
#   df[col+"_d"]=df[col].dt.day
#   if col2!=0:
#     df[col2+"_y"]=df[col2].dt.year
#     df[col2+"_m"]=df[col2].dt.month
#     df[col2+"_d"]=df[col2].dt.day
#     df[col+"-"+col2]=df[col] - df[col2]
#   return df

# def time_encode(df, col):
#     # この方法だと場合によって最大値が変化するデータでは正確な値は出ない
#     # 例：月の日数が30日や31日の場合がある
#     df[col + '_cos'] = np.cos(2 * np.pi * df[col] / df[col].max())
#     df[col + '_sin'] = np.sin(2 * np.pi * df[col] / df[col].max())
#     return df


In [ ]:
# 数値カラムの外れ値の影響を減らすためのクリッピング
class clip_numeric(base):
  def __init__(self,col):
    # 数値コラムでなければエラーを出す
    if df_train[col].dtype == "float64" or df_train[col].dtype == "int64":
      self.col = col 
    else:
      raise ValueError("col is not numeric")
    
  def transform(self, input_df):
    out_df = pd.DataFrame()
    upperbound, lowerbound= np.percentile(input_df[self.col],[1,99])
    out_df[self.col]=np.clip(input_df[self.col],upperbound,lowerbound)
    assert len(out_df)==len(input_df)
    return out_df.add_suffix("_clip")

test = clip_numeric("work_year")
test.transform(df_train)

In [ ]:
#カテゴリ列について、trainingにしか現れないカテゴリ、もしくはtestにしか現れないカテゴリを可視化
from matplotlib_venn import venn2
import matplotlib.pyplot as plt

def check_cat(df_train, df_test, col):
  train_s = set(df_train[col].unique())
  test_s = set(df_test[col].unique())
  fig, ax = plt.subplots(figsize=(6, 4))
  venn2(subsets=(train_s, test_s), set_labels=('train', 'test'),ax=ax)
  ax.set_title(col)
  common = train_s & test_s
  print(len(common))
  # print(common)
  return ax

check_cat(df_train, df_test, "job_title")

In [ ]:
#カテゴリ特徴量について、トレーニングに一定数以上いない物をまとめてOthersに落とす。
class drop_minor_cat_to_others(base):
  def __init__(self,col):
    self.col = col
    self.thresh = 10
    
  def fit(self, input_df):
    output_df = pd.DataFrame()
    value_count = input_df[self.col].value_counts()
    apply_values = value_count[value_count < self.thresh].index
    output_df[self.col] = input_df[self.col].where(~(input_df[self.col].isin(apply_values)),"Others")
    self.apply_values_ = apply_values
    return output_df
  
  def transform(self, input_df):
    output_df = pd.DataFrame()
    output_df[self.col] = input_df[self.col].where(~(input_df[self.col].isin(self.apply_values_)),"Others")
    return output_df

test = drop_minor_cat_to_others("job_title")
test.fit(df_train)

In [ ]:
# 主にコンペ用。One_hot_encodingではトレーニングデータとテストデータに共通しない値をOthersに落とす。周辺の特徴量を用いて汎用的な値に直すことも検討
class drop_only_train_test_values_to_others(base):
  def __init__(self, col):
    self.col = col
  
  def transform(self, input_df):
    output_df = pd.DataFrame()
    train_values = set(df_train[self.col].unique())
    test_values = set(df_test[self.col].unique())
    common_values = train_values & test_values
    output_df[self.col] = input_df[self.col].where(input_df[self.col].isin(common_values),"Others")
    return output_df
  
test = drop_only_train_test_values_to_others("job_title")
test.fit(df_train)

In [ ]:
# カテゴリのone_hot_encoding用。コンペ用の変換。
# TrainとTestに共通に出現する値のみでone_hot_encodingを行う。
# 実務であればTrain側のカテゴリを保有して予測時に適用する形に編集する
class one_hot_encoding(base):
    def __init__(self, col):
        self.col = col
        train_values = df_train[self.col].unique()
        test_values = df_test[self.col].unique()
        self.common_values = set(train_values) & set(test_values)
    
    def transform(self, input_df):
        x = input_df[self.col]
        cat = pd.Categorical(x, categories=self.common_values)
        out_df = pd.get_dummies(cat)
        return out_df
    


test = one_hot_encoding("job_title")
test.fit(df_train)
    

In [ ]:
# テキストの特徴量に対してElastic-netでstacked predictionでメタ特徴量を作成する。
# 分類にするならLogisticRegressionのペナルティを編集してElastic-netに。https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
# Foldを変えるならはここを参照 https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection
# 線形モデルを変えるならここを参照 https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model
# メトリクスを変えるならここを参照https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, KFold
from scipy.sparse import hstack
from sklearn.linear_model import ElasticNet,LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

class text_out_of_fold_prediction(base):
  def __init__(self, col):
    self.nfolds = 5
    self.seed = 71
    self.target = "salary"
    self.col = col
    self.models = []
    self.regression = True
    
  def fit(self, input_df):
    text_df = input_df[self.col].fillna("nan")
    self.tfidf_ = TfidfVectorizer(max_features=10000)
    converted_txt = self.tfidf_.fit_transform(text_df)
    if self.regression:
      fold = KFold(n_splits=self.nfolds, random_state=self.seed, shuffle=True)
    else:
      fold = StratifiedKFold(n_splits=self.nfolds, random_state=self.seed, shuffle=True)
    oof_train = np.zeros(len(input_df))
    scores = []
    for i, (train_idx, valid_idx) in enumerate(fold.split(input_df[self.col], input_df[self.target])):
      train_x, train_y = converted_txt[train_idx], input_df.loc[train_idx, self.target]
      valid_x, valid_y = converted_txt[valid_idx], input_df.loc[valid_idx, self.target]
      
      if self.regression:
        clf = ElasticNet(random_state=0)
        clf.fit(train_x, train_y)
        pred_y = clf.predict(valid_x)
        score = mean_squared_error(valid_y, pred_y)
      else:
        clf = LogisticRegression(penalty='elasticnet',solver='sag')
        clf.fit(train_x, train_y)
        pred_y = clf.predict_proba(valid_x)[:,1]
        score = roc_auc_score(valid_y, pred_y)  
      print(f'CV Score of Fold_{i} is {score}')
      self.models.append(clf)
      scores.append(score)
      oof_train[valid_idx]= pred_y
    print(f"mean score is {np.mean(scores)}")
    output_df = pd.DataFrame()
    output_df[self.col] = oof_train
    return output_df.add_prefix("predicted_")
        
  def transform(self, input_df):
    text_df = input_df[self.col].fillna("nan")
    converted_txt = self.tfidf_.transform(text_df)
    oof_test = np.zeros(len(input_df))
    if self.regression:
      for clf in self.models:
        oof_test += clf.predict(converted_txt)
    else:
      for clf in self.models:
        oof_test += clf.predict_proba(converted_txt)[:,1]
    oof_test /= self.nfolds
    output_df = pd.DataFrame()
    output_df[self.col] = oof_test
    return output_df.add_prefix("predicted_")

test = text_out_of_fold_prediction("job_title")
test.fit(df_train)
test.transform(df_test)

In [ ]:
# # カテゴリ特徴量の変換　https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/
# # https://contrib.scikit-learn.org/category_encoders/
# import category_encoders as ce
# import warnings
# warnings.simplefilter('ignore')

# def cat_encode_for_liner(df, df2, col):
#   encode = ce.OneHotEncoder(cols = col, handle_unknown = 'error', use_cat_names = True) #初回は初見のカテゴリを調べるために"error"で。
#   df_tmp = encode.fit_transform(df[col])
#   df = pd.concat([df, df_tmp], axis = 1)
#   df.drop(columns = col, inplace = True)
#   df_tmp = encode.transform(df2[col])
#   df2 = pd.concat([df2, df_tmp], axis = 1)
#   df2.drop(columns = col, inplace = True)
#   return df, df2

# for col in lis_cols_cat:
#   df_train, df_test = cat_encode_for_liner(df_train, df_test, col)
# print(df_train.shape, df_test.shape)
# df_train.head()

In [ ]:
##　要リファクタリング
# #欠損値補完の前に欠損値カウント
# def null_count(df):
#   df["null_count"] = df.isnull().sum(axis=1)
#   return df

# df_train = null_count(df_train)
# df_test = null_count(df_test)
# df_train.head()

In [ ]:
## 要修正
# #数値型特徴量の欠損値補完を行う。欠損値がある列には欠損フラグの列も追加 #純粋な欠損値だけ見ている。データを確認し、偽装欠損値と見込まれる値が見つかれば別途対応
# from sklearn.impute import SimpleImputer

# def impute_and_flag(df_train, df_test, col_lis):
#   imp_lis = []
#   for col in col_lis:
#     if df_train[col].isnull().any() == True:
#       imp_lis.append(col)
#       df_train[col+"_null"] = 0
#       df_train.loc[df_train[col].isnull(), col+"_null"] = 1
#       df_test[col+"_null"] = 0
#       df_test.loc[df_test[col].isnull(), col+"_null"] = 1
#       imp = SimpleImputer(missing_values = np.nan, strategy = 'median')
#       df_train[col] = imp.fit_transform(df_train[[col]])
#       df_test[col] = imp.transform(df_test[[col]])
#     elif df_train[col].isnull().any() == True:
#       print("missing values is found in only test data for : "+ col)
#       #欠損値がTestデータにだけ現れるケースはデータを要確認。そもそもその列は使えないかも。
#     else:
#       pass
#   print("Imputed :"+ str(imp_lis))
#   return df_train, df_test


# df_train, df_test = impute_and_flag(df_train, df_test, lis_cols_num)

In [ ]:
#数値型特徴量の標準化　https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing
from sklearn.preprocessing import StandardScaler

class standardize(base):
  def __init__(self,col):
    self.col = col
    self.scaler = StandardScaler()
  
  def fit(self,input_df):
    output_df = pd.DataFrame()
    output_df[self.col] = self.scaler.fit_transform(input_df[[self.col]]).ravel()
    return output_df.add_prefix("std_")
  
  def transform(self, input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = pd.DataFrame()
    output_df[self.col] = self.scaler.transform(input_df[[self.col]]).ravel()
    return output_df.add_prefix("std_")

test = standardize("remote_ratio")
test.fit(df_train)

In [ ]:
df_train.head()

In [ ]:
df_train

In [ ]:
feature_blocks = [*[drop_minor_cat_to_others(col) for col in
                    ['work_year', 'experience_level', 'employment_type', 'job_title',
       'employee_residence','company_location', 'company_size']],
                 *[drop_only_train_test_values_to_others(col) for col in ['work_year', 'experience_level', 'employment_type', 'job_title',
       'employee_residence','company_location', 'company_size']],
                 *[one_hot_encoding(col) for col in ['work_year', 'experience_level', 'employment_type', 'job_title',] 
                  ],
                 *[standardize(col) for col in ["remote_ratio"]]
                 ]

def run_blocks(input_df, blocks, test=False):
    output_df = pd.DataFrame()
    for block in blocks:
        output_i = block.fit(input_df)
        assert len(output_i) == len(input_df)
        print(output_i.shape)
        output_df = pd.concat([output_df, output_i], axis=1)
    return output_df

run_blocks(df_train, blocks=feature_blocks)


In [ ]:
#パーティショニングとStacked prediction
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
# Foldを変えるならはここを参照 https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection
# 線形モデルを変えるならここを参照 https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model
# メトリクスを変えるならここを参照https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

class linear_model_cv_prediction(base):
  def __init__(self, col):
    self.nfolds = 5
    self.seed = 71
    self.target = "salary"
    self.models = []
    self.regression = True
    self.train_folds = []
    self.valid_folds = []
    # elasticnet
    self.params = [alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=0, selection='cyclic', tol=0.0001, warm_start=False]
    
    # logistic regression
    # self.params = [C=1.0, class_weight=None, dual=False, fit_intercept=True,
    #                intercept_scaling=1, l1_ratio=0.5, max_iter=100,
    #                multi_class='auto', n_jobs=None, penalty='elasticnet',
    #                random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
    #                warm_start=False]
    
  def fit(self, input_df):
    if self.regression:
      fold = KFold(n_splits=self.nfolds, random_state=self.seed, shuffle=True)
    else:
      fold = StratifiedKFold(n_splits=self.nfolds, random_state=self.seed, shuffle=True)
    
    oof_train = np.zeros(len(input_df))
    scores = []
    for i, (train_idx, valid_idx) in enumerate(fold.split(input_df, input_df[self.target])):
      train_x, train_y = input_df[train_idx], input_df.loc[train_idx, self.target]
      valid_x, valid_y = input_df[valid_idx], input_df.loc[valid_idx, self.target]
      
      self.train_folds.append(train_idx)
      self.valid_folds.append(valid_idx)
      
      if self.regression:
        clf = ElasticNet(**self.params)
        clf.fit(train_x, train_y)
        pred_y = clf.predict(valid_x)
        score = mean_squared_error(valid_y, pred_y)
      else:
        clf = LogisticRegression(penalty='elasticnet',solver='sag')
        clf.fit(train_x, train_y)
        pred_y = clf.predict_proba(valid_x)[:,1]
        score = roc_auc_score(valid_y, pred_y)  
        
      print(f'CV Score of Fold_{i} is {score}')
      self.models.append(clf)
      scores.append(score)
      oof_train[valid_idx]= pred_y
    print(f"mean score is {np.mean(scores)}")
    output_df = pd.DataFrame()
    output_df[self.col] = oof_train
    return output_df.add_prefix("predicted_")
  
  def transform(self, input_df):
    text_df = input_df[self.col].fillna("nan")
    converted_txt = self.tfidf_.transform(text_df)
    oof_test = np.zeros(len(input_df))
    if self.regression:
      for clf in self.models:
        oof_test += clf.predict(converted_txt)
    else:
      for clf in self.models:
        oof_test += clf.predict_proba(converted_txt)[:,1]
    oof_test /= self.nfolds
    output_df = pd.DataFrame()
    output_df[self.col] = oof_test
    return output_df.add_prefix("predicted_")
  

df_train_pred, df_test_pred, score_table = linear_model_cv_prediction(df_train, df_train_y, df_test, fold, clf_lis)
score_table